# LSTM-FCN for time series classification

Project on [GitHub](https://github.com/titu1994/LSTM-FCN), [paper](https://ieeexplore.ieee.org/document/8141873/). Models work OK for univariate time series.

![LSTM FCN model](https://raw.githubusercontent.com/titu1994/LSTM-FCN/master/images/LSTM-FCN.png)

In [1]:
import os, sys
import pandas as pd
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns import dlbs_models as models

from tensorflow.python.keras import layers, models, regularizers

#import tensorflow as tf
#import tensorflow.contrib.keras as K
import numpy as np

In [2]:
# Problems are defined here:
# https://github.com/titu1994/LSTM-FCN/blob/master/utils/constants.py

# Problem 118: http://www.timeseriesclassification.com/description.php?Dataset=PigCVP
#     Train size: 104, test size: 208, length: 2000, num_classes: 52
MAX_SEQUENCE_LENGTH = 2000    # 118
NB_CLASSES =52                # 118
NUM_CELLS = 128               # Largest value in `all_datasets_training.py`

perf = []

# LSTM FCN

In [3]:
def generate_lstmfcn(max_sequence_length, nb_classes, num_cells=8):
    ip = layers.Input(shape=(1, max_sequence_length))

    x = layers.LSTM(num_cells)(ip)
    x = layers.Dropout(0.8)(x)

    y = layers.Permute((2, 1))(ip)
    y = layers.Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.GlobalAveragePooling1D()(y)

    x = layers.Concatenate()([x, y])

    out = layers.Dense(nb_classes, activation='softmax')(x)

    model = models.Model(ip, out, name='LSTM-FCN')

    return model

In [4]:
estimate(generate_lstmfcn(MAX_SEQUENCE_LENGTH, NB_CLASSES, NUM_CELLS), perf, perf)

W0523 04:39:25.647135 140196554352384 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f8172e482b0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Layer not recognized (type=<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>, name=input_1)


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(1, 2000)",0.000000,0,2000,0.000000,0.008000
1,permute,"(2000, 1)",0.000000,0,2000,0.000000,0.008000
2,conv1d,"(2000, 128)",0.002048,1152,256000,0.004608,1.024000
3,batch_normalization_v1,"(2000, 128)",0.000000,256,256000,0.001024,1.024000
4,activation,"(2000, 128)",0.000000,0,256000,0.000000,1.024000
5,conv1d_1,"(2000, 256)",0.327680,164096,512000,0.656384,2.048000
6,batch_normalization_v1_1,"(2000, 256)",0.000000,512,512000,0.002048,2.048000
7,activation_1,"(2000, 256)",0.000000,0,512000,0.000000,2.048000
8,conv1d_2,"(2000, 128)",0.196608,98432,256000,0.393728,1.024000
9,batch_normalization_v1_2,"(2000, 128)",0.000000,256,256000,0.001024,1.024000


In [5]:
column_oder = ['name', 'phase', 'input_shape', 'num_parameters', 'param_memory', 'flops', 'activation_memory']
columns = {'name': 'Model', 'phase': 'Phase', 'input_shape': 'Input shape', 'num_parameters': '#Parameters',
           'param_memory': 'Model size (MB) FP32', 'flops': 'GFLOPs (multiply-add)',
           'activation_memory': 'Activation size (MB) FP32'}
df = pd.DataFrame(perf, columns=column_oder)
df.rename(columns=columns)

,Model,Phase,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,LSTM-FCN,inference,"(1, 2000)",1369140,5.47656,0.527439,12.313632
1,LSTM-FCN,training,"(1, 2000)",1369140,5.47656,1.582317,24.627264
